In [ ]:
#| default_exp convTranspose_replace

In [ ]:
#| export
from segmentation_test.pytorch_model_development import *

In [ ]:
#| export
import torch 
import torch.nn as nn
from fastcore.test import *

In [ ]:
x = torch.randn(1, 1, 256, 256)
c1 = DoubleConv(1, 16)(x)
p1 = Pooling(max_pool_only=True)(c1)
c2 = DoubleConv(16, 32)(p1)
p2 = Pooling(max_pool_only=True)(c2)
c3 = DoubleConv(32, 64)(p2)
p3 = Pooling(max_pool_only=True)(c3)
c4 = DoubleConv(64, 128)(p3)
p4 = Pooling(max_pool_only=True)(c4)
c5 = DoubleConv(128, 256)(p4)

In [ ]:
print(f'c1 has a shape of ={c1.shape}')
print(f'c2 has a shape of ={c2.shape}')
print(f'c3 has a shape of ={c3.shape}')
print(f'c4 has a shape of ={c4.shape}')
print(f'c5 has a shape of ={c5.shape}')

c1 has a shape of =torch.Size([1, 16, 256, 256])
c2 has a shape of =torch.Size([1, 32, 128, 128])
c3 has a shape of =torch.Size([1, 64, 64, 64])
c4 has a shape of =torch.Size([1, 128, 32, 32])
c5 has a shape of =torch.Size([1, 256, 16, 16])


In [ ]:
#| export
class ConvTrns(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvTrns, self).__init__()
        self.up = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

    def forward(self, x):
        return self.up(x)

In [ ]:
up1 = ConvTrns(256, 128)(c5)
up1.shape

torch.Size([1, 128, 32, 32])

In [ ]:
#| export
class CustomConvTranspose(nn.Module):
    ' Custom conv transpose, need in_channels and out_channels'

    def __init__(self, in_channels, out_channels):
        super(CustomConvTranspose, self).__init__()
        self.up = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='nearest'),  # Resize layer
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)  # Conv layer
        )

    def forward(self, x):
        x = self.up(x)
        return x


In [ ]:
test_eq(ConvTrns(256, 128)(c5).shape, CustomConvTranspose(256, 128)(c5).shape)
test_eq(ConvTrns(128, 64)(c4).shape, CustomConvTranspose(128, 64)(c4).shape)

In [ ]:
#output_onnx='torch_simple.onnx'

In [ ]:
#torch.onnx.export(
    #model, 
    #dummy_input, 
    #output_onnx, 
    #export_params=True, 
    #opset_version=13, 
    #do_constant_folding=True, 
    #input_names=['input'], 
    #output_names=['output'], 
    ##dynamic_axes={
        ##'input': {0: 'batch_size'}, 
        ##'output': {0: 'batch_size'}
        ##}
        #)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export('11_convTranspose.ipynb')